## Versuch 2

In [1]:
from keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 200
WIDTH = 200

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

Using TensorFlow backend.
/home/dzeri/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dzeri/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dzeri/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dzeri/.local/lib/python3.6/site-packages/tensorflow/python/fra

/home/dzeri/.local/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [2]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "./persons-cropped/"
BATCH_SIZE = 10

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=45,
      horizontal_flip=False,
      vertical_flip=False
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE,
                                                   class_mode='categorical')

Found 3800 images belonging to 141 classes.


In [3]:

from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

#class_list = ["Pizza", "Burger", "Taco"]
FC_LAYERS = []
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=141)
finetune_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [6]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

NUM_EPOCHS = 10
num_train_images = 3800
#num_train_images = 25

adam = Adam()
sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
finetune_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="./checkpoints/" + "ResNet50" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True, callbacks=callbacks_list)

Epoch 1/10
380/380 [==============================] - 270s 711ms/step - loss: 128.8047 - accuracy: 0.2045

Epoch 00001: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 2/10
380/380 [==============================] - 269s 709ms/step - loss: 59.4051 - accuracy: 0.4005

Epoch 00002: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 3/10
380/380 [==============================] - 265s 697ms/step - loss: 36.6674 - accuracy: 0.5468

Epoch 00003: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 4/10
380/380 [==============================] - 264s 694ms/step - loss: 25.9998 - accuracy: 0.6482

Epoch 00004: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 5/10
380/380 [==============================] - 265s 697ms/step - loss: 20.7497 - accuracy: 0.7042

Epoch 00005: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 6/10
380/380 [==============================] - 264s 695ms/step - loss: 15.8305 - accuracy: 0.7539

Epoch 00006: savin

In [15]:
# Plot the training and validation loss + accuracy
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    # plt.figure()
    # plt.plot(epochs, loss, 'r.')
    # plt.plot(epochs, val_loss, 'r-')
    # plt.title('Training and validation loss')
    plt.show()

    plt.savefig('acc_vs_epochs.png')
    
plot_training(history)

KeyError: 'acc'